In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/Parkinson

In [ ]:
import os
import pandas as pd
from collections import defaultdict

pd.options.display.max_rows = 100
pd.options.display.max_columns = 50

In [ ]:
# Rawdata가 저장되어 있는 경로
DATASETPATH = os.path.join(os.getcwd(), 'Rawdata')
DATA_CONTROL = os.path.join(DATASETPATH , 'Controls')  # '/content/drive/MyDrive/Parkinson/Rawdata/Controls'
DATA_PD = os.path.join(DATASETPATH, 'PD')              # '/content/drive/MyDrive/Parkinson/Rawdata/PD'

# 추출 후 저장할 경로
PREPPATH = os.path.join(os.getcwd(), 'Prep')           
PREP_CONTROL = os.path.join(PREPPATH , 'Controls')    # '/content/drive/MyDrive/Parkinson/Prep/Controls'
PREP_PD = os.path.join(PREPPATH , 'PD')               # '/content/drive/MyDrive/Parkinson/Prep/Controls'

if not os.path.exists(PREPPATH):
    os.mkdir(PREPPATH)

if not os.path.exists(PREP_CONTROL):
    os.mkdir(PREP_CONTROL)

if not os.path.exists(PREP_PD):
    os.mkdir(PREP_PD)

### 39개 관측 센서 위치

In [ ]:
BODY = ['LFHD', 'RFHD', 'LBHD', 'RBHD',  # 머리 앞뒤
        'CLAV',  # 쇄골
        'RBAK',  # 어깨뼈
        'C7',    # 경추 7번
        'STRN',  # 흉골(명치)
        'T10',   # 흉추 10번
        ]

ARMS = ['LSHO', 'RHSO',  # 어깨
        'LUPA', 'RUPA',  # 위팔
        'LELB', 'RELB',  # 팔꿈치
        'LFRM', 'RFRM',  # 아래팔
        'LWRA', 'RWRA', 'LWRB', 'RWRB',  # 손목
        'LFIN', 'RFIN',  # 손가락
        ]

LEGS = ['LASI', 'RASI', 'LPSI', 'RPSI',  # 골반 앞뒤
        'LTHI', 'RTHI',  # 대퇴
        'LKNE', 'RKNE',  # 무릎
        'LTIB', 'RTIB',  # 정강이
        'LHEE', 'RHEE',  # 발뒤꿈치
        'LANK', 'RANK',  # 복사뼈
        'LTOE', 'RTOE',  # 발가락
        ]

ALL = BODY + ARMS + LEGS
len(ALL)

## 데이터 추출
- Subframe, NaN 열 제거
- 두번째 행의 측정 단위를 보면 369개의 측정값 중 1/3은 거리(mm), 1/3은 속도(mm/s), 나머지 1/3은 가속도(mm/s^2)임을 알 수 있다. \
따라서, 속도와 가속도 데이터를 제외한 123개의 거리 피처만 추출한다.
- CentreOfMass_XYZ, CentreOfMasFloor_XYZ 제거
- 총 컬럼 수: 센서 39개 * 3 (X,Y,Z) = 117개

In [ ]:
def prepData(target_path, target_file, save_path, print_filename=False):
    """
    target_path: './Rawdata/Controls' or './Rawdata/PD'
    target_file: '이니셜_FW1.csv', '이니셜_FW_01.csv', 'KangYK_FW01.csv'
    save_path: './Prep/Controls' or './Prep/PD'
    """
    # 저장 파일명: '이니셜_FW/BW1~3.csv'
    splitted = target_file[:-4].split('_')
    if len(splitted) == 2 and len(splitted[1]) > 3:
        splitted[1] = splitted[1][:2] + str(int(splitted[1][2:]))
    elif len(splitted) == 3:
        splitted[1] = splitted[1] + str(int(splitted[-1]))
        splitted = splitted[:2]

    save_filename = '_'.join(splitted) + '.csv'  # 데이터 추출 후 저장할 파일명

    ptype = target_path.split('/')[-1]
    if print_filename:
        print(f'Processing for {ptype}/{target_file}')

    # 첫 두 행(Trajectories 정보) 제외하고 읽기
    df = pd.read_csv(os.path.join(target_path, target_file), skiprows=2, index_col=0)

    # 첫번쨰 열 SubFrame 제거
    df.drop(columns='Unnamed: 1', inplace=True)

    # 마지막 열 NaN 제거
    if df.iloc[:,-1].isna().all():
        df.drop(columns=df.columns[-1], inplace=True)

    # 단위가 mm인 데이터 추출
    df = df[df.columns[df.iloc[1]=='mm']]

    # CentreOfMass_XYZ, CentreOfMasFloor_XYZ 등 39개 센서 이외 변수 제거
    if len(df.columns) > len(ALL) * 3:
        df = df.iloc[:, :len(ALL)-1]  # 마지막 컬럼 6개 제거
    elif len(df.columns) < len(ALL) * 3:
        print(f'Check the columns in {ptype} / {target_file}')
        print(f'Number of columns : {len(df.columns)} (Goal: 117)')

    # 컬럼 이름 변경
    col_names = []
    for i, col in enumerate(df.columns):
        position_name = col.split(':')[1]
        col_names.append(position_name + '_' + df.iloc[0,i])  # LFHD_X, LFHD_Y, LFHD_Z, ...
    df.columns = col_names

    # Frame(XYZ), 단위(mm) 행 제거
    df = df.iloc[2:] 

    # Prep 폴더 생성 후 csv로 저장하기
    if not os.path.exists(save_path):
        os.mkdir(save_path)
    df.to_csv( os.path.join(save_path, save_filename) , encoding='utf-8', index=True)

    return

In [ ]:
def runPrepData(rawdata_path, save_path):
    for filename in os.listdir(rawdata_path):
        # if filename.endswith('.csv'):  # --> 제대로 작동 안하는 듯!
        prepData(rawdata_path, filename, save_path, print_filename=False)

In [ ]:
runPrepData(DATA_CONTROL, PREP_CONTROL)
runPrepData(DATA_PD, PREP_PD)

In [ ]:
print(f"Raw_PD: {len(os.listdir(DATA_PD))} -> Prep_PD: {len(os.listdir(PREP_PD))}")
print(f"Raw_Cont: {len(os.listdir(DATA_CONTROL))} -> Prep_Cont: {len(os.listdir(PREP_CONTROL))}")

> Controls는 왜 개수가 줄어들었지???? \
  --> endswith('.csv')가 제대로 작동 안 하는 듯!!



## 환자 정보 취합
- 환자 이름: 영문 이니셜
- 소속: PD, Controls
- FW, BW 개수
- FW, BW 파일 위치

In [ ]:
def getPatientsTable(target_path):
    """
    target_path: './Rawdata/Controls' or './Rawdata/PD'
    """
    
    # 환자별 FW, BW 파일 개수 세기
    category = target_path.split('/')[-1]  # 소속: PD, Controls
    cntFW = defaultdict(int)
    cntBW = defaultdict(int)
    pathFW = defaultdict(list)
    pathBW = defaultdict(list)

    for filename in sorted(os.listdir(target_path)):
        if not filename.endswith('.csv'): continue
            
        splitted = filename[:-4].split('_')  # [파일명 이니셜, FW1]
        name = splitted[0]  # 환자 이름 (이니셜, 영문명)
        
        if splitted[1][:2] == 'FW':
            cntFW[name] += 1
            pathFW[name].append( './Prep/' + category + '/' + filename ) 
        elif splitted[1][:2] == 'BW':
            cntBW[name] += 1
            pathBW[name].append( './Prep/' + category + '/' + filename)
    
    # FW만 있는 환자는 BW 0으로 입력
    for k in cntFW.keys():
        if k not in cntBW.keys():
            cntBW[k] = 0
            pathBW[k] = []
    
    # 딕셔너리 정렬: 키 값(이니셜) 기준
    cntFW = dict(sorted(cntFW.items()))
    cntBW = dict(sorted(cntBW.items()))
    pathFW = dict(sorted(pathFW.items()))
    pathBW = dict(sorted(pathBW.items()))

    df = pd.DataFrame()
    df['Patient'] = list(cntFW.keys())
    df['Category'] = category
    df['cntFW'] = list(cntFW.values())
    df['cntBW'] = list(cntBW.values())
    df['pathFW'] = pathFW.values()
    df['pathBW'] = pathBW.values()
    
    return df

In [ ]:
df_PD = getPatientsTable(DATA_PD)
df_control = getPatientsTable(DATA_CONTROL)
df_info = pd.concat([df_PD, df_control], ignore_index=True)

df_info.to_csv(os.path.join(os.getcwd(), 'csv_info.csv'), encoding='utf-8', index=False)

## 수진쌤 코드 터미널 실행

In [ ]:
%cd /content/drive/MyDrive/NIMS/Parkinson/code/utils

In [ ]:
!bash python preprocessor.py --debug=true